In [68]:
import psycopg2
import pandas as pd
import numpy as np
import mariadb
import json
import os
import shutil
import subprocess
from pathlib import Path
import pyodbc

In [124]:
water_df = pd.read_csv("input/water_analysis_cleaned.csv")

In [125]:
water_df.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'sample_code', 'batch_date',
       'analysis_name', 'chemical_name', 'result', 'unit_name'],
      dtype='object')

In [134]:
water_df.loc[water_df['analysis_name'].str.contains('BTEX')]

,Unnamed: 0.1,Unnamed: 0,sample_code,batch_date,analysis_name,chemical_name,result,unit_name
14156,14156,14156,CR027WA0040,2011-12-19 00:00:00.000,BTEX Analysis,xylene_,1.0,ug/l
14157,14157,14157,CR027WA0040,2011-12-19 00:00:00.000,BTEX Analysis,toluene_,1.0,ug/l
14158,14158,14158,CR027WA0040,2011-12-19 00:00:00.000,BTEX Analysis,benzene__,1.0,ug/l
14159,14159,14159,CR027WA0040,2011-12-19 00:00:00.000,BTEX Analysis,ethyl_benzene,1.0,ug/l
14160,14160,14160,CR027WA0041,2011-12-19 00:00:00.000,BTEX Analysis,xylene_,1.0,ug/l
14161,14161,14161,CR027WA0041,2011-12-19 00:00:00.000,BTEX Analysis,toluene_,1.0,ug/l
14162,14162,14162,CR027WA0041,2011-12-19 00:00:00.000,BTEX Analysis,benzene__,1.0,ug/l
14163,14163,14163,CR027WA0041,2011-12-19 00:00:00.000,BTEX Analysis,ethyl_benzene,1.0,ug/l
14164,14164,14164,CR027WA0042,2011-12-19 00:00:00.000,BTEX Analysis,xylene_,1.0,ug/l
14165,14165,14165,CR027WA0042,2011-12-19 00:00:00.000,BTEX Analysis,toluene_,1.0,ug/l


In [135]:
water_df['analysis_name'] = [ str(i).split(",") for i in water_df['analysis_name'] ]

In [136]:
water_df.head()

,Unnamed: 0.1,Unnamed: 0,sample_code,batch_date,analysis_name,chemical_name,result,unit_name
0,0,0,CA335WA0010,2019-04-18 00:00:00.000,[Reverse Osmosis Water Use],ph,8.27,NaN
1,1,1,CD026WA2057,2019-05-07 00:00:00.000,[Reverse Osmosis Water Use],ph,8.25,NaN
2,2,2,CB012WA0052,2012-07-19 00:00:00.000,"[Irrigation Water Analysis, Feacal Coliforms (...",ph,7.25,NaN
3,3,3,CB012WA0052,2012-07-19 00:00:00.000,"[Irrigation Water Analysis, Feacal Coliforms (...",electrical_conductivity,0.15,mS cm -1
4,4,4,CB012WA0052,2012-07-19 00:00:00.000,"[Irrigation Water Analysis, Feacal Coliforms (...",ammonium,0.01,ppm


In [137]:
_df = water_df.explode("analysis_name")
_df.to_csv("output/water_single_analysis.csv")

In [138]:
_df

,Unnamed: 0.1,Unnamed: 0,sample_code,batch_date,analysis_name,chemical_name,result,unit_name
0,0,0,CA335WA0010,2019-04-18 00:00:00.000,Reverse Osmosis Water Use,ph,8.270,NaN
1,1,1,CD026WA2057,2019-05-07 00:00:00.000,Reverse Osmosis Water Use,ph,8.250,NaN
2,2,2,CB012WA0052,2012-07-19 00:00:00.000,Irrigation Water Analysis,ph,7.250,NaN
2,2,2,CB012WA0052,2012-07-19 00:00:00.000,Feacal Coliforms (MPN) and E.Coli,ph,7.250,NaN
3,3,3,CB012WA0052,2012-07-19 00:00:00.000,Irrigation Water Analysis,electrical_conductivity,0.150,mS cm -1
...,...,...,...,...,...,...,...,...
1240310,1240318,1240318,CA028WA0130,2024-05-21 14:27:00.000,Standard Drinking Water Analysis (WHO),nitrate_n,5.900,ppm
1240311,1240319,1240319,CB387WA0006,2024-05-21 09:32:00.000,Standard Drinking Water Analysis (WHO),phosphorus,0.012,ppm
1240312,1240320,1240320,CB387WA0006,2024-05-21 09:32:00.000,Standard Drinking Water Analysis (WHO),sulphur,5.270,ppm
1240313,1240321,1240321,CB387WA0006,2024-05-21 09:32:00.000,Standard Drinking Water Analysis (WHO),iron,0.013,ppm


In [139]:
_df = pd.read_csv("output/water_single_analysis.csv")

In [152]:
_df.loc[_df['analysis_name'].str.contains('BTEX')]

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,sample_code,batch_date,analysis_name,chemical_name,result,unit_name
21200,14156,14156,14156,CR027WA0040,2011-12-19 00:00:00.000,BTEX Analysis,xylene_,1.0,ug/l
21201,14157,14157,14157,CR027WA0040,2011-12-19 00:00:00.000,BTEX Analysis,toluene_,1.0,ug/l
21202,14158,14158,14158,CR027WA0040,2011-12-19 00:00:00.000,BTEX Analysis,benzene__,1.0,ug/l
21203,14159,14159,14159,CR027WA0040,2011-12-19 00:00:00.000,BTEX Analysis,ethyl_benzene,1.0,ug/l
21204,14160,14160,14160,CR027WA0041,2011-12-19 00:00:00.000,BTEX Analysis,xylene_,1.0,ug/l
21205,14161,14161,14161,CR027WA0041,2011-12-19 00:00:00.000,BTEX Analysis,toluene_,1.0,ug/l
21206,14162,14162,14162,CR027WA0041,2011-12-19 00:00:00.000,BTEX Analysis,benzene__,1.0,ug/l
21207,14163,14163,14163,CR027WA0041,2011-12-19 00:00:00.000,BTEX Analysis,ethyl_benzene,1.0,ug/l
21208,14164,14164,14164,CR027WA0042,2011-12-19 00:00:00.000,BTEX Analysis,xylene_,1.0,ug/l
21209,14165,14165,14165,CR027WA0042,2011-12-19 00:00:00.000,BTEX Analysis,toluene_,1.0,ug/l


In [153]:
analysisDetails = pd.read_csv("output/analysis_chemicals.csv")

In [154]:
analysisDetails.analysis_name = [ i.replace(":","") for i in analysisDetails.analysis_name ]

In [155]:
analysisDetails['chemical_name'] = [ i.replace("'","").replace("[","").replace("]","").split(",") for i in analysisDetails['chemical_name'] ]

In [156]:
analysisDetails

,analysis_name,chemical_name
0,% Gypsum,[%_gypsum]
1,% Nitrogen (RSSP 2),[total_nitrogen]
2,% Organic Matter,[organic_matter]
3,% Organic Matter (RSSP 2),[organic_matter]
4,% Soil Nitrogen,[total_nitrogen]
...,...,...
1300,"pH, EC & Sodium Analysis","[ph, sodium, electrical_conductivity]"
1301,"pH, EC & Sodium Analysis (Substrate)","[ph, ph, ph, ph, sodium, electrical_condu..."
1302,"pH, EC & Sodium Analysis (drain)","[ph, sodium, electrical_conductivity]"
1303,paraquat+diquat,"[cec, paraquat+diquat_, paraquat+diquat_]"


In [157]:
analysisDetails = analysisDetails.set_index("analysis_name")

In [158]:
analysisDetails = analysisDetails.to_dict()['chemical_name']

In [159]:
analysisDetails['Basic Drinking Water Analysis'] = analysisDetails['Basic Drinking Water Analysis (WHO)'] 

In [160]:
analysisDetails['BTEX Analysis']

['benzene__', ' toluene_', ' xylene_', ' ethyl_benzene']

In [166]:
_df['chemical_name'] =_df['chemical_name'].str.strip()

In [171]:
water_df = pd.DataFrame()
for analysis in _df.analysis_name.unique():
    # print(analysis)
    # print(analysisDetails[analysis])
    # print(_df.loc[_df['chemical_name'].isin([j.strip() for j in analysisDetails[analysis]])].chemical_name.unique())
    if analysis in analysisDetails.keys():
        _ = _df.loc[(_df['analysis_name'] == analysis) & (_df['chemical_name'].isin([j.strip() for j in analysisDetails[analysis]]))]
        water_df = pd.concat([water_df,_])

In [172]:
water_df.loc[water_df['analysis_name'].str.contains('BTEX')]

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,sample_code,batch_date,analysis_name,chemical_name,result,unit_name
21200,14156,14156,14156,CR027WA0040,2011-12-19 00:00:00.000,BTEX Analysis,xylene_,1.0,ug/l
21201,14157,14157,14157,CR027WA0040,2011-12-19 00:00:00.000,BTEX Analysis,toluene_,1.0,ug/l
21202,14158,14158,14158,CR027WA0040,2011-12-19 00:00:00.000,BTEX Analysis,benzene__,1.0,ug/l
21203,14159,14159,14159,CR027WA0040,2011-12-19 00:00:00.000,BTEX Analysis,ethyl_benzene,1.0,ug/l
21204,14160,14160,14160,CR027WA0041,2011-12-19 00:00:00.000,BTEX Analysis,xylene_,1.0,ug/l
21205,14161,14161,14161,CR027WA0041,2011-12-19 00:00:00.000,BTEX Analysis,toluene_,1.0,ug/l
21206,14162,14162,14162,CR027WA0041,2011-12-19 00:00:00.000,BTEX Analysis,benzene__,1.0,ug/l
21207,14163,14163,14163,CR027WA0041,2011-12-19 00:00:00.000,BTEX Analysis,ethyl_benzene,1.0,ug/l
21208,14164,14164,14164,CR027WA0042,2011-12-19 00:00:00.000,BTEX Analysis,xylene_,1.0,ug/l
21209,14165,14165,14165,CR027WA0042,2011-12-19 00:00:00.000,BTEX Analysis,toluene_,1.0,ug/l


In [173]:
# _df.to_csv("output/water_single_analysis.csv",chunksize=1000)

In [174]:
# water_df = _df.copy(deep=True)

In [175]:
np.unique([ i for i in water_df.analysis_name if "Reverse" in i])

array(['Reverse Osmosis Water Use'], dtype='<U25')

In [176]:
np.unique([ i for i in water_df.analysis_name if "Dist" in i])

array([], dtype=float64)

In [177]:
np.unique([ i for i in water_df.analysis_name if "(FAO)" in i])

array(['Irrigation Water Analysis (FAO)'], dtype='<U31')

In [178]:
np.unique([ i for i in water_df.analysis_name if "Fruit" in i])

array([], dtype=float64)

In [179]:
np.unique([ i for i in water_df.analysis_name if "Reagent" in i])

array(['Reagent Water Analysis'], dtype='<U22')

In [180]:
np.unique([ i for i in water_df.analysis_name if "Tea" in i])

array(['Tea Extract Analysis'], dtype='<U20')

In [181]:
irrigation_analysis_names = np.unique([ i for i in water_df.analysis_name if "Irrigation" in i])

In [182]:
irrigation_analysis_names

array(['Basic Irrigation Water Analysis',
       'Irrigation Water Analysis (FAO)',
       'Standard Irrigation Water Analysis'], dtype='<U34')

In [183]:
analysis_freq = water_df.analysis_name.value_counts()

In [184]:
analysis_freq = analysis_freq.loc[analysis_freq.values >= 820]

In [185]:
# water_df = water_df.loc[water_df.analysis_name.isin(analysis_freq.index)]

In [186]:
water_df

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,sample_code,batch_date,analysis_name,chemical_name,result,unit_name
0,0,0,0,CA335WA0010,2019-04-18 00:00:00.000,Reverse Osmosis Water Use,ph,8.27,NaN
1,1,1,1,CD026WA2057,2019-05-07 00:00:00.000,Reverse Osmosis Water Use,ph,8.25,NaN
41,22,22,22,CA335WA0010,2019-04-18 00:00:00.000,Reverse Osmosis Water Use,electrical_conductivity,0.99,mS cm -1
42,23,23,23,CA335WA0010,2019-04-18 00:00:00.000,Reverse Osmosis Water Use,nitrate_n,0.63,ppm
43,24,24,24,CA335WA0010,2019-04-18 00:00:00.000,Reverse Osmosis Water Use,nitrates,2.79,ppm
...,...,...,...,...,...,...,...,...,...
1558463,1098798,1098806,1098806,CC029WA0384,2023-12-15 12:22:35.000,Lite Drinking Water Analysis,ph,6.82,NaN
1558464,1098799,1098807,1098807,CC029WA0384,2023-12-15 12:22:35.000,Lite Drinking Water Analysis,electrical_conductivity,0.27,mS cm -1
1558465,1098800,1098808,1098808,CC029WA0384,2023-12-15 12:22:35.000,Lite Drinking Water Analysis,fluoride,0.48,ppm
1558466,1098801,1098809,1098809,CC029WA0384,2023-12-15 12:22:35.000,Lite Drinking Water Analysis,e_coli,0.00,mpn/100ml


In [187]:
# water_df.loc[water_df.analysis_name.isin(irrigation_analysis_names), "analysis_name"] = "Irrigation"

In [188]:
water_df = water_df.loc[water_df.analysis_name != 'Tea Extract Analysis']

In [189]:
water_df = water_df.loc[~(water_df['result'] == "ND") ]
water_df = water_df.loc[~(water_df['result'] == "Detected") ]

In [190]:
water_df

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,sample_code,batch_date,analysis_name,chemical_name,result,unit_name
0,0,0,0,CA335WA0010,2019-04-18 00:00:00.000,Reverse Osmosis Water Use,ph,8.27,NaN
1,1,1,1,CD026WA2057,2019-05-07 00:00:00.000,Reverse Osmosis Water Use,ph,8.25,NaN
41,22,22,22,CA335WA0010,2019-04-18 00:00:00.000,Reverse Osmosis Water Use,electrical_conductivity,0.99,mS cm -1
42,23,23,23,CA335WA0010,2019-04-18 00:00:00.000,Reverse Osmosis Water Use,nitrate_n,0.63,ppm
43,24,24,24,CA335WA0010,2019-04-18 00:00:00.000,Reverse Osmosis Water Use,nitrates,2.79,ppm
...,...,...,...,...,...,...,...,...,...
1558463,1098798,1098806,1098806,CC029WA0384,2023-12-15 12:22:35.000,Lite Drinking Water Analysis,ph,6.82,NaN
1558464,1098799,1098807,1098807,CC029WA0384,2023-12-15 12:22:35.000,Lite Drinking Water Analysis,electrical_conductivity,0.27,mS cm -1
1558465,1098800,1098808,1098808,CC029WA0384,2023-12-15 12:22:35.000,Lite Drinking Water Analysis,fluoride,0.48,ppm
1558466,1098801,1098809,1098809,CC029WA0384,2023-12-15 12:22:35.000,Lite Drinking Water Analysis,e_coli,0.00,mpn/100ml


In [191]:
# water_df = water_df.dropna(subset="result")

In [192]:
# water_df['result'] = [ float(i.replace("<","").replace(">","").strip()) for i in water_df['result']]

In [193]:
water_df['unit_name'] = [ str(i).strip() for i in water_df['unit_name'].values ]

In [194]:
# water_df['chemical_name'] = [ str(i).lower().replace(" ","_").replace("(","").replace(")","").replace(".","").strip() for i in water_df['chemical_name'].values ]

In [195]:
# water_df = water_df.loc[~(water_df['result'] < 0)]

In [196]:
df_ = water_df[['analysis_name','chemical_name','unit_name']].value_counts()
df_.to_csv("output/water_unit_per_chemical.csv")

In [197]:
df_ = df_.reset_index()

In [198]:
water_df

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,sample_code,batch_date,analysis_name,chemical_name,result,unit_name
0,0,0,0,CA335WA0010,2019-04-18 00:00:00.000,Reverse Osmosis Water Use,ph,8.27,nan
1,1,1,1,CD026WA2057,2019-05-07 00:00:00.000,Reverse Osmosis Water Use,ph,8.25,nan
41,22,22,22,CA335WA0010,2019-04-18 00:00:00.000,Reverse Osmosis Water Use,electrical_conductivity,0.99,mS cm -1
42,23,23,23,CA335WA0010,2019-04-18 00:00:00.000,Reverse Osmosis Water Use,nitrate_n,0.63,ppm
43,24,24,24,CA335WA0010,2019-04-18 00:00:00.000,Reverse Osmosis Water Use,nitrates,2.79,ppm
...,...,...,...,...,...,...,...,...,...
1558463,1098798,1098806,1098806,CC029WA0384,2023-12-15 12:22:35.000,Lite Drinking Water Analysis,ph,6.82,nan
1558464,1098799,1098807,1098807,CC029WA0384,2023-12-15 12:22:35.000,Lite Drinking Water Analysis,electrical_conductivity,0.27,mS cm -1
1558465,1098800,1098808,1098808,CC029WA0384,2023-12-15 12:22:35.000,Lite Drinking Water Analysis,fluoride,0.48,ppm
1558466,1098801,1098809,1098809,CC029WA0384,2023-12-15 12:22:35.000,Lite Drinking Water Analysis,e_coli,0.00,mpn/100ml


In [199]:
analysis_chem_unit_df = pd.DataFrame()
for analysis in df_['analysis_name'].unique():
    print(analysis)
    chems_count_ = df_.loc[df_['analysis_name'] == analysis]
    duplicates_chem = (chems_count_[chems_count_.duplicated(subset='chemical_name')])
    temp_analysis_chem_unit_df = pd.DataFrame()
    for index,row in duplicates_chem.iterrows():
        chemical = row['chemical_name']
        chems_count_2 = chems_count_.loc[chems_count_['chemical_name'] == chemical ]
        unit = chems_count_2.loc[chems_count_2['count'] == max(chems_count_2['count'])]['unit_name'].values[0]
        _ = pd.DataFrame({"analysis_name":[analysis],"chemical_name":[chemical], "unit_name":[unit]})
        temp_analysis_chem_unit_df = pd.concat([temp_analysis_chem_unit_df, _])
    analysis_chem_unit_df = pd.concat([analysis_chem_unit_df, temp_analysis_chem_unit_df])
analysis_chem_unit_df.to_csv("output/water_unit_per_chemical_decision.csv")

Standard Drinking Water Analysis (WHO)
Irrigation Water Analysis (FAO)
Standard Drinking Water Analysis
Basic Drinking Water Analysis
Reverse Osmosis Water Use
Basic Drinking Water Analysis (WHO)
Heavy Metals in Water
Base Titanium Water Analysis
E coli & Total Coliforms
Post Harvest Water Analysis
Total Suspended Solids
Total Bacteria Count
Total Dissolved Solids
KEBS Drinking Water Standard Analysis
KEBS Drinking Water Microbiology
Standard Drinking Water + Free Chlorine Analysis
Turbidity Analysis
Feacal Coliforms (MPN) and E.Coli
Total Coliforms and Faecal E.Coli
Free Chlorine
Water Colour Analysis
Total Chlorine
Taita Water Analysis (Uni of Helsinki)
WRMA Water Analysis
Water Borne Diseases
Arsenic in Water
Aquasearch Full Chemical Analysis
Total Bacterial Count
Legionella
Alkalinity in Water
pH in Water
E coli
KEBS Drinking Water Minimum Monitoring
Chemical & Biochemical Oxygen Demand
Oil & Grease
Total coliforms
Lead in Water
Iodine Analysis
Biological Oxygen Demand
Selected ele

In [200]:
analysis_chem_unit_df

,analysis_name,chemical_name,unit_name
0,Standard Drinking Water Analysis (WHO),total_coliforms,mpn/100ml
0,Standard Drinking Water Analysis (WHO),e_coli,mpn/100ml
0,Standard Drinking Water Analysis (WHO),calcium,ppm
0,Standard Drinking Water Analysis (WHO),zinc,ppm
0,Standard Drinking Water Analysis (WHO),manganese,ppm
...,...,...,...
0,BTEX Analysis,ethyl_benzene,ug/l
0,BTEX Analysis,benzene__,ug/l
0,BTEX Analysis,xylene_,ug/l
0,BTEX Analysis,toluene_,ug/l


In [201]:
df = pd.DataFrame()
for index, row in analysis_chem_unit_df.iterrows():
    analysis = row['analysis_name']
    chemical = row['chemical_name']
    unit = row['unit_name']
    temp_df = water_df.loc[(water_df['analysis_name'] == analysis) & (water_df['chemical_name'] == chemical) & (water_df['unit_name'] == unit)]
    df = pd.concat([df, temp_df])

In [202]:
df

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,sample_code,batch_date,analysis_name,chemical_name,result,unit_name
12499,8921,8921,8921,CC064WA0109,2023-05-24 14:19:00.000,Standard Drinking Water Analysis (WHO),total_coliforms,0.0,mpn/100ml
16538,11427,11427,11427,CP207WA0005,2020-05-18 10:41:03.000,Standard Drinking Water Analysis (WHO),total_coliforms,0.0,mpn/100ml
24488,16269,16269,16269,CK391WA0001,2020-05-18 11:53:00.000,Standard Drinking Water Analysis (WHO),total_coliforms,4.0,mpn/100ml
25157,16767,16767,16767,CH006WA1475,2020-09-23 12:21:00.000,Standard Drinking Water Analysis (WHO),total_coliforms,26.0,mpn/100ml
27641,18681,18681,18681,CM507WA0001,2020-05-18 15:55:00.000,Standard Drinking Water Analysis (WHO),total_coliforms,180.0,mpn/100ml
...,...,...,...,...,...,...,...,...,...
24600,16345,16345,16345,CR027WA0186,2012-03-15 00:00:00.000,BTEX Analysis,toluene_,1.0,ug/l
91666,63166,63166,63166,CR027WA0039,2011-12-19 00:00:00.000,BTEX Analysis,toluene_,1.0,ug/l
59853,40706,40706,40706,CS001WA0005,2014-10-30 00:00:00.000,Salmonella and E.Coli (Water),e_coli,1.0,cfu/100 ml
342158,230285,230285,230285,CC003WA0072,2014-10-28 00:00:00.000,Salmonella and E.Coli (Water),e_coli,1.0,cfu/100 ml


In [203]:
df.to_csv("output/water_analysis_cleaned.csv",chunksize=1000)

In [204]:
df[["analysis_name", "chemical_name", "unit_name"]].set_index('analysis_name').to_csv('output/expected_units/analysis_units.csv')

In [40]:
# df[["analysis_name", "chemical_name", "unit_name"]].set_index('analysis_name').to_dict('dict') 

In [40]:
water_df = pd.read_csv("output/water_analysis_cleaned.csv")

In [41]:
water_df = water_df[['sample_code','analysis_name','chemical_name', 'result']]

In [42]:
water_df['analysis_name'] = [ i.split(",") for i in water_df['analysis_name'] ]

In [43]:
water_df['analysis_name']

0                                   [Irrigation]
1                                   [Irrigation]
2                                   [Irrigation]
3                                   [Irrigation]
4                                   [Irrigation]
                           ...                  
1006750    [KS Drinking Water Standard Analysis]
1006751    [KS Drinking Water Standard Analysis]
1006752    [KS Drinking Water Standard Analysis]
1006753    [KS Drinking Water Standard Analysis]
1006754    [KS Drinking Water Standard Analysis]
Name: analysis_name, Length: 1006755, dtype: object

In [45]:
water_df.analysis_name = [ i[0] for i in water_df.analysis_name ] 

In [48]:
water_df.chemical_name.value_counts()

bicarbonate               48654
ammonium                  48458
zinc                      46425
copper                    46419
boron                     46418
                          ...  
dissolved_silica             24
total_viable_count_@37       17
sodium_chloride               4
escherichia_coli              2
moisture                      2
Name: chemical_name, Length: 70, dtype: int64

In [46]:
df.to_csv("output/water_analysis_cleaned.csv",chunksize=1000)

In [ ]:
# _df = pd.DataFrame()
# for index, row in water_df.iterrows():
#     analyses = row['analysis_name']
#     sample_code = row['sample_code']
#     for analysis in analyses:
#         _ = pd.DataFrame(row).T
#         _['analysis_name'] = analysis
#         _df = pd.concat([_df, _])

# _df.to_csv("output/water_single_analysis.csv")

In [36]:
water_df = pd.read_csv("output/water_analysis_cleaned.csv")

In [37]:
water_df_pivoted = pd.pivot_table(data=water_df, index="sample_code", columns="chemical_name", values="result")

In [39]:
water_df_pivoted.columns

Index(['aluminium', 'ammonium', 'arsenic', 'barium', 'bicarbonate', 'bod',
       'boron', 'cadmium', 'calcium', 'carbonate', 'charge_balance',
       'chloride', 'chlorides', 'chromium', 'cobalt', 'cod',
       'colloidal_silica', 'colour', 'copper', 'dissolved_silica', 'e_coli',
       'electrical_conductivity', 'escherichia_coli', 'faecal_e_coli',
       'feacal_coliforms', 'feacal_coliforms_mpn', 'feacal_e_coli',
       'fluorides', 'hardness', 'iodine', 'iron', 'lead', 'magnesium',
       'manganese', 'mercury', 'moisture', 'molybdenum', 'nickel', 'nitrate_n',
       'nitrates', 'nitrite', 'oil_&_grease', 'ph', 'phosphate', 'phosphorus',
       'potassium', 'pseudomonas', 'salmonella', 'sar', 'selenium', 'shigella',
       'silica', 'silicon__', 'silver', 'sodium', 'sodium_chloride',
       'strontium', 'sulphate', 'sulphates', 'sulphur', 'tbc_at_22_degrees',
       'tbc_at_37_degrees', 'total_coliforms', 'total_dissolved_solids',
       'total_suspended_solids', 'total_viable_cou

In [43]:
water_df_pivoted.total_suspended_solids.info()

<class 'pandas.core.series.Series'>
Index: 20522 entries, AGD102-124WA0001 to I48-1-9WA0001
Series name: total_suspended_solids
Non-Null Count  Dtype  
--------------  -----  
2564 non-null   float64
dtypes: float64(1)
memory usage: 836.7+ KB
